In [17]:
import sqlite3
import pandas as pd


#### Creamos nuestra conexión a la B.D

In [73]:
conn = sqlite3.connect('problemas_seleccion.db')
df = pd.read_sql('SELECT * FROM TABLA_VENTAS WHERE TIENDA = 1 AND ID_CATEGORIA = 5', conn)


#### Revisando los datos

Inspeccionemos un poco los tipos de nuestras variables y nuestro DataFrame

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TIENDA        852 non-null    int64  
 1   ID_CATEGORIA  852 non-null    int64  
 2   FECHA         852 non-null    object 
 3   VENTAS        852 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 26.8+ KB


In [75]:
df.describe()

,TIENDA,ID_CATEGORIA,VENTAS
count,852.0,852.0,852.000000
mean,1.0,5.0,2287.154957
std,0.0,0.0,1605.793504
min,1.0,5.0,362.000000
25%,1.0,5.0,1086.000000
50%,1.0,5.0,1799.688996
75%,1.0,5.0,3008.363742
max,1.0,5.0,10878.000000


Notamos que nuestra columna FECHA está en formato de string, entonces, lo ideal es transformarla a Datetime para poder trabajar nuestro Forecast. Además nos debemos deshacer de las columnas TIENDA e ID_CATEGORIA que en este momento no están aportando

In [76]:
df['FECHA'] = pd.to_datetime(df['FECHA'])
df = df.drop(columns = ['TIENDA', 'ID_CATEGORIA'], axis = 1)

Inspeccionemos nuestros datos:

In [77]:
df.head()

,FECHA,VENTAS
0,2018-02-02,2636.0
1,2018-05-16,818.0
2,2019-05-22,910.0
3,2018-10-11,1387.0
4,2018-02-14,1137.0


Ordenamos las fechas:

In [78]:
df = df.sort_values(by='FECHA')
df = df.reset_index()

In [80]:
df = df.drop('index', axis = 1)

In [58]:
df.drop('index', axis = 1)

,FECHA,VENTAS
0,2017-08-01,1142.000000
1,2017-08-02,1079.000000
2,2017-08-03,1096.000000
3,2017-08-04,2430.000000
4,2017-08-05,4261.000000
...,...,...
847,2019-12-11,1290.206994
848,2019-12-12,1387.618000
849,2019-12-13,2535.910981
850,2019-12-14,4388.873985


#### Pequeña exploración

Para fines de exploración de los datos descompondremos la fecha:

In [82]:
df_exploratorio = df.copy()
df_exploratorio['dia'] = df_exploratorio.FECHA.map(lambda x: x.day)
df_exploratorio['mes'] = df_exploratorio.FECHA.map(lambda x: x.month)
df_exploratorio['anio'] = df_exploratorio.FECHA.map(lambda x: x.year)


In [83]:
df_exploratorio.head()

,FECHA,VENTAS,dia,mes,anio
0,2017-08-01,1142.0,1,8,2017
1,2017-08-02,1079.0,2,8,2017
2,2017-08-03,1096.0,3,8,2017
3,2017-08-04,2430.0,4,8,2017
4,2017-08-05,4261.0,5,8,2017


Observemos los datos!

In [113]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_exploratorio['FECHA'], y=df_exploratorio['VENTAS'], mode='lines', fill='tozeroy', fillcolor='#c6ccd8',
                     marker=dict(color= '#496595'), name='Ventas por Día'))
fig.update_layout(height=750, bargap=0.15,
                  margin=dict(b=0,r=20,l=20), 
                  title_text="Ventas por Día",
                  template="plotly_white",
                  title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'),
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"),
                  showlegend=False)
fig.show()

Haremos una descomposición de la fecha para ver el comportamiento general de las ventas:

In [94]:


df_2017 = df_exploratorio[df_exploratorio['anio']==2017][['mes','VENTAS']]
df_2017 = df_2017.groupby('mes').agg({"VENTAS" : "mean"}).reset_index().rename(columns={'VENTAS':'s17'})
df_2018 = df_exploratorio[df_exploratorio['anio']==2018][['mes','VENTAS']]
df_2018 = df_2018.groupby('mes').agg({"VENTAS" : "mean"}).reset_index().rename(columns={'VENTAS':'s18'})
df_2019 = df_exploratorio[df_exploratorio['anio']==2019][['mes','VENTAS']]
df_2019 = df_2019.groupby('mes').agg({"VENTAS" : "mean"}).reset_index().rename(columns={'VENTAS':'s19'})


In [95]:
df_2017.columns

Index(['mes', 's17'], dtype='object')

In [91]:
print(df_2017.shape, df_2018.shape,df_2019.shape)

(5, 2) (12, 2) (12, 2)


Observemos que el df_2017 tiene sólo los últimos 5 meses como dato, si la muestra hubiese tenido 7 o más meses hubiesemos podido reemplazar los meses faltantes con la media de los 7 meses, pero como no es el caso y solo para fines de visualización obviaremos este para tener un panorama más general.

In [96]:

df_year = df_2018.merge(df_2019,on='mes')


top_labels = ['2018', '2019']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)']

# X axis value 
df_year = df_year[['s18','s19']].replace(np.nan,0)
x_data = df_year.values

# y axis value (Month)
df_2018['mes'] =['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dec']
y_data = df_2018['mes'].tolist()

fig = go.Figure()
for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        fig.add_trace(go.Bar(
            x=[xd[i]], y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(color='rgb(248, 248, 249)', width=1)
            )
        ))

fig.update_layout(title='Media de Ventas por año',
    xaxis=dict(showgrid=False, 
               zeroline=False, domain=[0.15, 1]),
    yaxis=dict(showgrid=False, showline=False,
               showticklabels=False, zeroline=False),
    barmode='stack', 
    template="plotly_white",
    margin=dict(l=0, r=50, t=100, b=10),
    showlegend=False, 
)

annotations = []
for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                          showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]
fig.update_layout(
    annotations=annotations)
fig.show()

Como primera visualización podemos notar que el efecto Fiestas Patrias, Navidad y Año Nuevo realmente hacen una diferencia. Analicemos mas en detalle nuestras ventas:

In [97]:
import calendar
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
offline.init_notebook_mode(connected = True)

df_exploratorio['week'] = df_exploratorio['FECHA'].dt.isocalendar().week
df_exploratorio['quarter'] = df_exploratorio['FECHA'].dt.quarter
df_exploratorio['day_of_week'] = df_exploratorio['FECHA'].dt.day_name()

df_m_sa = df_exploratorio.groupby('mes').agg({"VENTAS" : "mean"}).reset_index()
df_m_sa['VENTAS'] = round(df_m_sa['VENTAS'],2)
df_m_sa['month_text'] = df_m_sa['mes'].apply(lambda x: calendar.month_abbr[x])
df_m_sa['text'] = df_m_sa['month_text'] + ' - ' + df_m_sa['VENTAS'].astype(str) 

df_w_sa = df_exploratorio.groupby('week').agg({"VENTAS" : "mean"}).reset_index() 
df_q_sa = df_exploratorio.groupby('quarter').agg({"VENTAS" : "mean"}).reset_index() 
# chart color
df_m_sa['color'] = '#496595'
df_m_sa['color'][:-1] = '#c6ccd8'
df_w_sa['color'] = '#c6ccd8'

# chart
fig = make_subplots(rows=2, cols=2, vertical_spacing=0.08,
                    row_heights=[0.7, 0.3], 
                    specs=[[{"type": "bar"}, {"type": "pie"}],
                           [{"colspan": 2}, None]],
                    column_widths=[0.7, 0.3],
                    subplot_titles=("Análisi de Venta Mensual", 'Análisis de Venta Cuatrimestral', 
                                    "Análisis de Ventas Semanal"))

fig.add_trace(go.Bar(x=df_m_sa['VENTAS'], y=df_m_sa['mes'], marker=dict(color= df_m_sa['color']),
                     text=df_m_sa['text'],textposition='auto',
                     name='Mes', orientation='h'), 
                     row=1, col=1)
fig.add_trace(go.Pie(values=df_q_sa['VENTAS'], labels=df_q_sa['quarter'], name='Cuarto',
                     marker=dict(colors=['#334668','#496595','#6D83AA','#91A2BF','#C8D0DF']), hole=0.7,
                     hoverinfo='label+percent+value', textinfo='label+percent'), 
                     row=1, col=2)
fig.add_trace(go.Scatter(x=df_w_sa['week'], y=df_w_sa['VENTAS'], mode='lines+markers', fill='tozeroy', fillcolor='#c6ccd8',
                     marker=dict(color= '#496595'), name='Week'), 
                     row=2, col=1)

# styling
fig.update_yaxes(visible=False, row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_xaxes(tickmode = 'array', tickvals=df_w_sa.week, ticktext=[i for i in range(1,53)], 
                 row=2, col=1)
fig.update_yaxes(visible=False, row=2, col=1)
fig.update_layout(height=750, bargap=0.15,
                  margin=dict(b=0,r=20,l=20), 
                  title_text="Análisis de Venta Media",
                  template="plotly_white",
                  title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'),
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"),
                  showlegend=False)
fig.show()

Nuevamente observamos que:

- Diciembre y Septiembre son los meses de mayores ventas, quitando esas observaciones del mapa podemos ver cierto patrón en la gráfica mensual, algo así como una curva normal. El efecto 'fiestas' es muy notorio.

- El segundo semestre del año es el periodo donde se debe tener mas stock de carne.

- A simple vista, las semanas que se escapan de la media de ventas son 9, 18, 19, 30, 31, 36, 37, 38, 43, 44, 51 y 52. Notemos que las semanas 36,37,38, 51 y 52 son semanas que dado los gráficos anteriores ya sabemos que se tendrá una alta demanda pues pertenecen a los meses de Sep y Dec respectivamente, entonces consideremos las otras: 

9: 1 Mar~7 Mar: Requiere mayor investigación, podría ser alguna promoción del supermercado.

18-19: 3 May~16 May -> Se puede deber a la preparación para el día 21 de mayo, feriado legal

30: 26 Jul~1 Ago: Requiere mayor investigación, podría ser alguna promoción del supermercado.

31: 2 Ago~8 Ago -> Se puede deber a la preparacion al proximo feriado, el día 15 de agosto.

43-44: 25 Oct~7 Nov -> Representan un periodo en el que hubieron feriados legales, además recordar que el 18 Oct del 2019 fue el Estallido Social lo que pudo haber generado en la población necesidad por abastecerse. 




Veamos ahroa que información podemos obtener del día:

In [105]:
df_dw_sa = df_exploratorio.groupby('day_of_week').agg({"VENTAS" : "mean"}).reset_index()
df_dw_sa['day_of_week']= df_dw_sa['day_of_week'].replace({'Monday': 'Lunes', 'Tuesday': 'Martes',
 'Wednesday': 'Miercoles', 'Thursday': 'Jueves', 'Friday': 'Viernes', 'Saturday': 'Sabado', 'Sunday': 'Domingo'})
df_dw_sa.VENTAS = round(df_dw_sa.VENTAS, 2)

# chart
fig = px.bar(df_dw_sa, y='day_of_week', x='VENTAS', title='Ventas Media vs Día de la Semana',
             color_discrete_sequence=['#c6ccd8'], text='VENTAS',
             category_orders=dict(day_of_week=["Lunes","Martes","Miercoles","Jueves", "Viernes","Sabado","Domingo"]))
fig.update_yaxes(showgrid=False, ticksuffix=' ', showline=False)
fig.update_xaxes(visible=False)
fig.update_layout(margin=dict(t=60, b=0, l=0, r=0), height=350,
                  hovermode="y unified", 
                  yaxis_title=" ", template='plotly_white',
                  title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'),
                  hoverlabel=dict(bgcolor="#c6ccd8", font_size=13, font_family="Lato, sans-serif"))

Notoriamente los fines de semana se consume mas carne!

### Un poco de modelamiento:

Usaremos FBProphet por su simplicidad y facilidad para ingresar datos como por ejemplo feriados

,day_of_week,VENTAS
0,Friday,2825.29
1,Monday,1566.15
2,Saturday,4303.75
3,Sunday,3115.21
4,Thursday,1611.21
5,Tuesday,1238.36
6,Wednesday,1274.65


In [104]:
df_dw_sa

,day_of_week,VENTAS
0,Viernes,2825.29
1,Lunes,1566.15
2,Sabado,4303.75
3,Domingo,3115.21
4,Jueves,1611.21
5,Martes,1238.36
6,Miercoles,1274.65
